# Import

In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

from googletrans import Translator

# MyDataFrame Class

In [171]:
class MyDataFrame: 
    def __init__(self, df, translate=False, translate_first_level=False, white_space=False, drop_level=True):
        
        self.df = df
        self.translator = Translator()
        self.title = ''
        self.unit = ''
        self.footer = ''
        self.translate = translate
        self.translate_first_level = translate_first_level
        self.white_space= white_space
        self.drop_level = drop_level
        #self.ratio = (((self.df.iloc[:,-2]-self.df.iloc[:,-3])/self.df.iloc[:,-3])*100).round()
        #self.last_column = self.df.iloc[:,-1].replace('..', 0).round()
        
        self.drop_na()
        
        if self.df.index.nlevels > 1:
            self.title_unit_multiindex();
        
        if self.drop_level:
            self.drop_levels()
            
        #if (self.ratio == self.last_column).all():
        #   self.drop_last_column()
        
        
        if self.translate:
            if self.df.index.nlevels == 1:
                self.translate_index()
                
            
        if self.translate:
            if self.df.index.nlevels > 1:
                self.translate_multi_index()
        
        
        if self.white_space:
            self.replace_white_space()
            
            
    def title_unit_multiindex(self):
        self.title = self.df.columns[0][0]
        self.unit = self.df.columns[1][1]
            
    def drop_levels(self):
        """
        This method drops two column levels that contained the infos previously captured (table's title and unit)
        """     
        while self.df.columns.nlevels>1:
            self.df.columns = self.df.columns.droplevel(0)
            
            
    def drop_na(self):
        """
        This method drops all rows and columns that have all values equals to NaN.
        """  
        self.df.dropna(how = 'all', inplace = True)
        self.df.dropna(axis = 'columns', how = 'all', inplace = True)

    
    def translate_index(self):
        """
        This method translates the index of a DataFrame to English.
        """
        self.new_index = []
        for index in self.df.index:
            if index == 'Reinjeção':
                self.new_index.append('Reinjection')
            elif (index == 'Espírito Santo') or (index == 'Espirito_Santo'):
                self.new_index.append('Espirito_Santo')
            elif index == 'Amazonas':
                self.new_index.append('Amazonas')
            elif index == 'Alagoas':
                self.new_index.append('Alagoas')
            elif (index == 'Ceará') or (index == 'Ceara'):
                self.new_index.append('Ceara')
            elif (index == 'Rio Grande do Norte') or (index == 'Rio_Grande_do_Norte'):
                self.new_index.append('Rio_Grande_do_Norte')
            else:
                self.new_index.append(self.translator.translate(index).text)
        self.df.index = self.new_index
        
                        
    def translate_multi_index(self):
        """
        This method translates a MultiIndex DataFrame to English.
        """
        
        if self.translate_first_level == True:
            for i, num in enumerate(self.df.index):
                    for j in range(self.df.index.nlevels):       
                        if j==0:
                            if (self.df.index[i][j] == 'Espírito_Santo') or (self.df.index[i][j] == 'Espirito_Santo'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Espírito_Santo','Espirito_Santo'), level = j)
                            elif self.df.index[i][j] == 'Amazonas':
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Amazonas','Amazonas'), level = j)
                            elif self.df.index[i][j] == 'Alagoas':
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Alagoas','Alagoas'), level = j)
                            elif (self.df.index[i][j] == 'Ceará') or (self.df.index[i][j] == 'Ceara'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Ceará','Ceara'), level = j)
                            elif (self.df.index[i][j] == 'Rio Grande do Norte') or (self.df.index[i][j] == 'Rio_Grande_do_Norte'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(' ','_'), level = j)
                            else:
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(self.df.index[i][j], self.translator.translate(self.df.index[i][j]).text), level = j)
        
        for i, num in enumerate(self.df.index):
            for j in range(self.df.index.nlevels):
                if j==0:
                    pass
                if j==1:
                    if self.df.index[i][j] == 'Mar': # checks if one of the words that the translate package can not translate
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Mar','Offshore'), level = j)
                    elif self.df.index[i][j] == 'Terra': # checks if one of the words that the translate package can not translate
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Terra','Onshore'), level = j)
                    elif not isinstance(self.df.index[i][j], str):
                        pass
                    else:
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(self.df.index[i][j], self.translator.translate(self.df.index[i][j]).text), level = j)
        
        
    def replace_underscore(self):
        """
        This method replaces all underscore for white space.
        """
        if self.df.index.nlevels > 1: # tells how many level are
            for i, level in enumerate(range(self.df.index.nlevels)): # runs through levels
                #for j, value in enumerate(self.df.index.levels[i]): # runs through the level's value and replace white space for underline
                self.df.index = self.df.index.set_levels(self.df.index.levels[i].str.replace("_", " "), level = i)
        
        elif self.df.index.nlevels == 1:
            self.new_index = []
            for index in self.df.index:
                self.new_index.append(index.replace('_', ' '))
            self.df.index = self.new_index
            
            
    def drop_unnamed_column(self):
        """
        This method drops the last column if it is a ration between columns[-2] and columns[-3] .
        """            
        for i,name in enumerate(total_reserves.df.columns):
            if type(name) == str and name.startswith('Unnamed'):
                self.df = self.df.drop(self.df.columns[-1], axis=1)

# Load and Wrangling

### Total¹ Reserves of Natural Gas, by Location (Shore and Offshore), according to Federation² Units - 2010-2019

In [172]:
total_reserves = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_5.xls',  header = [0,2,3], index_col = [0,1]), translate=True,)

In [173]:
#dropping Unnamed column
total_reserves.drop_unnamed_column()

# Setting unit, title and footer
total_reserves.unit = '10⁶ m³'
total_reserves.title = 'Total¹ Reserves of Natural Gas, by Location (Shore and Offshore), according to Federation² Units - 2010-2019'
total_reserves.footer ='Notes:\n1. Reserves on 12/31 of the reference years.\n2. See the General Notes item on "Brazilian Oil and Natural Gas Reserves".\n\nSource:\nANP / SDP, according to ANP Resolution No. 47/2014.\n\n¹ Including reserves whose fields Development Plans are under analysis.\n² The reserves are fully appropriated to the state in which each field has its area mostly located.\n³ The Roncador and Frade field reserves are fully appropriated in the State of Rio de Janeiro, for simplification.\n⁴ The Sapinhoá field reserves are fully appropriated in the State of São Paulo for simplification.\n⁵ The reserves in the Caravela field are fully appropriated in the State of Paraná, for simplification.\n⁶ The Tubarão field reserves are fully appropriated in the State of Santa Catarina, for simplification.\n'

# Correcting Index Supscrit
total_reserves.df.index = total_reserves.df.index.set_levels(total_reserves.df.index.levels[0].str.replace('3','³'), level = 0)
total_reserves.df.index = total_reserves.df.index.set_levels(total_reserves.df.index.levels[0].str.replace('4','⁴'), level = 0)
total_reserves.df.index = total_reserves.df.index.set_levels(total_reserves.df.index.levels[0].str.replace('5','⁵'), level = 0)
total_reserves.df.index = total_reserves.df.index.set_levels(total_reserves.df.index.levels[0].str.replace('6','⁶'), level = 0)

In [175]:
total_reserves.df

2010         2011           2012  \
Total               NaN       824723.2222  906530.6250  918568.744479   
Subtotal            Onshore   117227.0756  118523.9164  140979.405510   
                    Offshore  707496.1466  788006.7086  777589.338969   
Amazonas            Onshore    94456.2943   95742.7805   89236.607701   
Maranhão            Onshore        0.0000       0.0000   29704.588918   
Ceará               Onshore        0.0000       0.0000       0.000000   
                    Offshore    1447.0109     993.2576     453.825274   
Rio Grande do Norte Onshore     2188.7258    2277.2275    3274.698434   
                    Offshore   11354.6855   12038.6400   10401.348605   
Alagoas             Onshore     4173.3458    4335.7954    4222.946555   
                    Offshore    1085.0791     981.4851     761.581879   
Sergipe             Onshore     1483.7181    1912.6821    1756.306837   
                    Offshore    4303.3330    4054.9647    5209.844818   
Bahia               Onshore    13379.3620   12510.5730   12055.517895   
                    Offshore   30746.4516   29073.5249   28059.276773   
Espírito Santo      Onshore      731.6895     919.2594     728.739170   
                    Offshore   87033.6847   77694.3402  103074.727321   
Rio de Janeiro³     Offshore  504641.5344  551842.4858  531125.011121   
São Paulo⁴          Offshore   62946.0060  107109.4087   94267.612565   
Paraná⁵             Onshore      813.9401     825.5985       0.000000   
                    Offshore    1261.0771    1289.5173    1307.931554   
Santa Catarina⁶     Offshore    2677.2843    2929.0843    2928.179059   

                                       2013           2014           2015  \
Total               NaN       839505.828677  859771.266519  745909.598497   
Subtotal            Onshore   116584.502060  100997.980118   86575.414520   
                    Offshore  722921.326617  758773.286402  659334.183977   
Amazonas            Onshore    86963.304124   74486.059396   51225.378088   
Maranhão            Onshore     8651.926596    8405.637236   17677.180558   
Ceará               Onshore        0.000000       6.570600       0.000013   
                    Offshore     741.638800     502.907578     255.940367   
Rio Grande do Norte Onshore     2548.927338    2209.704036    2108.822494   
                    Offshore    9088.218790    8224.919170    2480.262648   
Alagoas             Onshore     4335.349042    3757.148426    2907.716600   
                    Offshore     656.364427     583.409393     502.253362   
Sergipe             Onshore     1813.597175    1729.969541    1565.341565   
                    Offshore    4813.045188    4185.986279    4776.685762   
Bahia               Onshore    11553.313029    9452.474279   10324.102107   
                    Offshore   24742.758601   24036.375003   13944.709525   
Espírito Santo      Onshore      718.084756     950.416605     766.873096   
                    Offshore   91556.894290   90662.524080   78963.731705   
Rio de Janeiro³     Offshore  507841.346935  555349.991275  490571.901104   
São Paulo⁴          Offshore   79254.969675   75227.173624   67838.699504   
Paraná⁵             Onshore        0.000000       0.000000       0.000000   
                    Offshore    1298.040442       0.000000       0.000000   
Santa Catarina⁶     Offshore    2928.049469       0.000000       0.000000   

                                       2016           2017           2018  \
Total               NaN       636834.920174  608459.147019  569998.888059   
Subtotal            Onshore    76258.568968   78741.612353   83062.695891   
                    Offshore  560576.351207  529717.534666  486936.192167   
Amazonas            Onshore    38686.313838   40757.087244   41073.851358   
Maranhão            Onshore    20412.202124   20822.208328   25720.764998   
Ceará               Onshore        0.000000       0.000000       0.000000   
                    Offshore     509.988719   

In [176]:
for i, index in enumerate(total_reserves.df.index.levels[0]):
    total_reserves.df.index.levels[0][i].replace('1','¹')
    total_reserves.df.index.levels[0][i].replace('2','²')
    total_reserves.df.index.levels[0][i].replace('3','³')
    total_reserves.df.index.levels[0][i].replace('4','⁴')
    total_reserves.df.index.levels[0][i].replace('5','⁵')
    total_reserves.df.index.levels[0][i].replace('6','⁶')
    total_reserves.df.index.levels[0][i].replace('7','⁷')
    total_reserves.df.index.levels[0][i].replace('8','⁸')
    total_reserves.df.index.levels[0][i].replace('9','⁹')

### Evolution of Natural Gas Processing Capacity, According to Production Centers - 2010-2019

In [177]:
processing = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_30.xls',  header = [0,2,3], index_col = [0]))

In [178]:
# Setting unit, title and footer
processing.unit = '10³ m³/dia'
processing.title = 'Evolution of Natural Gas Processing Capacity, According to Production Centers - 2010-2019'
processing.footer = '¹ Volume in the gaseous state.\n² Includes the UPGNs (Natural Gas Production Units) in Catu and Bahia until 2013. From 2014, only includes Catu.'

### Proved Reserved

In [179]:
proved_reserves = MyDataFrame(pd.read_excel(r'data_set/table-1_2.xls', header = [0,2,3], index_col = [0,1]))

In [180]:
# Dropping Unnamed Column
proved_reserves.drop_unnamed_column()

for name in proved_reserves.df.columns:
    if type(name)==str and name.startswith('Unnamed'):
        print(name)

Unnamed: 12_level_2


In [181]:
proved_reserves.df

2010         2011           2012  \
Brazil              NaN       423003  459403.2990  459187.425652   
Subtotal            Onshore    68803   70576.7220   72374.939592   
                    Offshore  354200  388826.5770  386812.486060   
Amazonas            Onshore    55878   57454.5421   51816.022349   
Maranhão            Onshore     -          0.0000    7285.712918   
Ceará               Offshore     652     528.2066     386.578242   
Rio Grande do Norte Onshore     1418    1464.2603    2549.515862   
                    Offshore    8676    7645.3401    7296.900782   
Alagoas             Onshore     2391    2515.4725    2739.945289   
                    Offshore    1085     981.4851     761.581879   
Sergipe             Onshore     1039    1432.6170    1460.099035   
                    Offshore    2588    2323.1322    3421.662230   
Bahia               Onshore     7356    6843.8661    5988.332034   
                    Offshore   26161   23708.1683   24289.975467   
Espírito Santo      Onshore      587     716.8581     535.312105   
                    Offshore   44025   43630.9721   42589.737557   
Rio de Janeiro3     Offshore  220506  249983.8568  246438.385470   
São Paulo4          Offshore   49373   58882.3934   60336.173351   
Paraná5             Onshore      134     149.1059       0.000000   
                    Offshore     904     913.0787    1061.758783   
Santa Catarina6     Offshore     230     229.9437     229.732299   

                                       2013           2014           2015  \
Brazil              NaN       457960.408587  471094.528310  429957.687363   
Subtotal            Onshore    69711.281247   71209.759856   70898.575363   
                    Offshore  388249.127340  399884.768454  359059.111999   
Amazonas            Onshore    50522.319249   52382.746618   46661.677237   
Maranhão            Onshore     6990.217596    7770.499588   12747.681137   
Ceará               Offshore     458.249387     325.491774     255.940367   
Rio Grande do Norte Onshore     1682.498005    1361.556831    1696.878793   
                    Offshore    5613.632255    5253.685492    2256.649235   
Alagoas             Onshore     2480.190476    2005.645021    1525.978796   
                    Offshore     656.364427     583.409393     502.253362   
Sergipe             Onshore     1555.492027    1501.844957    1373.350649   
                    Offshore    3397.609472    2960.834551    1580.677145   
Bahia               Onshore     5912.199180    5594.854979    6336.554473   
                    Offshore   20374.186229   17970.912247   11948.820710   
Espírito Santo      Onshore      568.364715     592.611862     556.454279   
                    Offshore   42862.986840   43687.364201   36907.154077   
Rio de Janeiro3     Offshore  257192.477821  274684.618543  256206.525310   
São Paulo4          Offshore   56406.023991   54418.452253   49401.091793   
Paraná5             Onshore        0.000000       0.000000       0.000000   
                    Offshore    1057.653208       0.000000       0.000000   
Santa Catarina6     Offshore     229.943710       0.000000       0.000000   

                                       2016           2017           2018  \
Brazil              NaN       377406.264803  369431.883952  368910.743505   
Subtotal            Onshore    61865.056844   66137.727803   69839.462088   
                    Offshore  315541.207959  303294.156148  299071.281418   
Amazonas            Onshore    36198.064198   39188.462631   38891.194229   
Maranhão            Onshore    15772.479759   16516.218328   20672.464998   
Ceará               Offshore     257.611324     197.491929     353.805202   
Rio Grande do Norte Onshore     1656.573966    1599.120068    1599.208235   
                    Offshore    2164.444988    1909.612504    1849.741032   
Alagoas             Onshore     1295.394608    1159.847270     875.077823   
                    Offshore     456.203076     394.974267     341.389

In [ ]:
domestic_sale = MyDataFrame(pd.read_excel(r'data_set/table-1_25.xls', header = [0,2,3], index_col = [0]))
domestic_sale.drop_last_column()
domestic_sale.unit = '10⁶ m³'

In [ ]:
balance = MyDataFrame(pd.read_excel(r'data_set/table-1_26.xls', header = [0,2,3], index_col = [0]))
balance.drop_last_column()

In [ ]:
production = MyDataFrame(pd.read_excel(r'data_set/table-1_5.xls', header = [0,2,3], index_col = [0,1]))
production.drop_last_column()
production.unit='10⁶ m³'

In [ ]:
gas_type = MyDataFrame(pd.read_excel(r'data_set/table-1_6.xls', header = [0,2,3], index_col = [0,1]))
gas_type.drop_last_column()
gas_type.unit='10⁶ m³'

In [ ]:
avg_price = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_24.xls',  header = [0,2,3,4], index_col = [0]), drop_level=False)
avg_price.drop_levels()
avg_price.unit='US$/mil m³'

In [ ]:
energy_matrix = pd.read_csv(r'data_set/GeracaoFonte.csv')
energy_matrix.columns = ['ID', 'Energy Type', 'GWh', 'Month', 'Year', 'dthProx']
energy_matrix = pd.pivot_table(energy_matrix, values='GWh', index=['Energy Type'], columns=['Year', 'Month'])

In [ ]:
bra_ind = pd.read_csv(r'data_set/API_BRA_DS2_en_csv_v2_1221399.csv', skiprows=[0,1,2,3])